# Fourier filtering, phase lock-in, and strain mapping
This example notebook demonstrates the Fourier filtering, phase lock-in analysis, and strain mapping on an atomic resolution HAADF STEM image of a partially relaxed thin film with misfit dislocations. 

The method used is described in detail in [Goodge, B. H., El Baggari, I., Hong, S. S., Wang, Z., Schlom, D. G., Hwang, H. Y., & Kourkoutis, L. F. (2022). Disentangling coexisting structural order through phase lock-in analysis of atomic-resolution STEM data. Microscopy and Microanalysis, 28(2), 404-411.](https://doi.org/10.1017/S1431927622000125)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from tifffile import imread
import kemstem
%matplotlib widget
print(f'{kemstem.__version__=}')

# Data loading and preprocessing
We begin by loading an atomic resolution HAADF STEM image of our film, and performing some basic preprocessing before beginning analysis. Here, the image is cropped to a square for ease of analysis and the intensity is normalized between 0 and 1 for interpretability.

In [ ]:
filename = 'data/rI_1341 2.6 Mx B_C270_0_HAADF.tif'

image = imread(filename)
print(f'Image shape {image.shape}')
image = image[:min(image.shape),:min(image.shape)]
print(f'Cropped to {image.shape}')
image = kemstem.util.normalize(image)
print(f'Normalized to min: {image.min()}, max: {image.max()}')

fig,ax = plt.subplots(1,1,constrained_layout=True,figsize=(4,4))
ax.matshow(image,cmap='gray')
ax.axis('off')

The doubling of peaks in the below pattern along both the in-plane and out-of-plane directions indicates that both lattice parameters differ between the film and substrate

In [ ]:
pattern_c = kemstem.fourier.prepare_fourier_pattern(image)
pattern_log = kemstem.fourier.prepare_fourier_pattern(image,log=True,log_offset=1e0) # Here we also take a log transform of the FFT for ease of visualization

fig,ax = plt.subplots(1,2,constrained_layout=True,figsize=(8,4))
ax[0].matshow(image,cmap='gray')
ax[0].axis('off')
ax[1].matshow(pattern_log,cmap='gray')
ax[1].axis('off')

# Identifying FFT peaks of interest
Here we can select one low order in-plane and one out-of-plane peak in the FFT to analyze the relaxation of the film.

The next cell allows these peaks to be picked manually, or uncomment the following cell to use a preselected set of peak positions.

In [ ]:
selected_peaks = kemstem.fourier.select_peaks(pattern_log,zoom=300,select_conjugates=False,figsize=(5,5),delete_within=5)

In [ ]:
# uncomment and run to use preselected peak(s)
#selected_peaks = np.array([[ 982.17262837,  825.96902674],
#       [1197.99177694,  980.77114342]])

# Fitting FFT peaks
Next, we fit the peaks using 2D gaussians. Here only the peak positions are important, so just refining them with e.g. center of mass is also reasonable.

These peak positions are quite important as they will essentially serve as without further adjustment they will serve as the reference parameters for the following phase and strain measurements. It may be interesting to adjust the peak guess positions and/or fit parameters to test the effect of mapping the film peak, the substrate peak, and the position between the two.

In [ ]:
p0 = np.array(selected_peaks).T
print(f'Selected X positions: {p0[:,1]}')
print(f'Selected Y positions: {p0[:,0]}')

peaks_ref,errs,opts,data_fits  = kemstem.fourier.refine_peaks_gf(gaussian_filter(pattern_log,1), p0, window_dimension=9,store_fits=True, remove_unfit = False)
fig,ax = plt.subplots(1,1,constrained_layout=True,figsize=(5,5))
kemstem.util.plot_numbered_points(pattern_log,p0,ax=ax,color='b',zoom=300)
ax.plot(peaks_ref[:,1],peaks_ref[:,0],'r.')
ax.axis('off')

_ = kemstem.util.plot_fit_comparison(data_fits,)

# Fourier Filtering
One of the peaks picked and refined previously can be selected for filtering with the `peak_index` variable. For filtering, the key parameter in addition to the peak position is the width of the gaussian profile applied around the peak in the FFT to select it - this width will be inversely proportional to the resulting resolution or coarsening length of the fourier filtered signal and its derivatives (the extracted phase and strain maps), but choosing too large of a filter will incorporate harmful noise in the filtered signal. This is set with the `sigma` variable that determines the standard deviation of the gaussian profile in k-space. The real space coarsening length corresponding to this filter size is visualized with the black and white coarsening markers in the bottom right of the images. 

In [ ]:
peak_index = 0
sigma= 10
filtered_im, filtered_ft, mask = kemstem.fourier.fourier_filter(pattern_c,peaks_ref[peak_index,:],sigma=sigma)

fig,ax = plt.subplots(1,3,constrained_layout=True,figsize=(12,4))
ax[0].matshow(image,cmap='gray')
ax[0].matshow(np.abs(filtered_im),cmap='inferno',alpha=.5)
ax[0].axis('off')
ax[1].matshow(np.real(filtered_im),cmap='gray')
ax[1].axis('off')
ax[2].matshow(pattern_log,cmap='gray')
ax[2].matshow(mask,alpha=.5,cmap='inferno')
ax[2].axis('off')

kemstem.util.coarsening_marker(ax[0],kemstem.fourier.coarsening_length(image.shape[0],sigma),edgecolor='k',facecolor='w')
kemstem.util.coarsening_marker(ax[1],kemstem.fourier.coarsening_length(image.shape[0],sigma),edgecolor='k',facecolor='w')

# Phase lock-in
The phase of the fourier filtered signal is extracted following the approach described in [Goodge, et al. (2020)](https://doi.org/10.1017/S1431927622000125). As noted above this will depend strongly on the precise peak position used for the fourier filtering and the fourier filter width. An additional minor parameter is the additional low-pass filtered used in the lock-in process, this typically should not need to be fine tuned but may need adjustment to avoid artifacts. Usually setting this to e.g. 2x the fourier filter sigma is sufficient. 

In [ ]:
phase = kemstem.fourier.phaselock(filtered_im,peaks_ref[peak_index,:],sigma=40) # different from the sigma above 
# this sigma is a parameter of the lock in analysis and should generally be larger than the sigma used for Fourier filtering
# but should not need to be changed here

fig,ax = plt.subplots(1,3,constrained_layout=True,figsize=(12,4))

ax[0].matshow(image,cmap='gray')
ax[0].axis('off')
ax[1].matshow(np.abs(filtered_im),cmap='inferno')
ax[1].axis('off')
kemstem.util.plot_phase(phase,ax=ax[2])
ax[2].axis('off')

kemstem.util.coarsening_marker(ax[1],kemstem.fourier.coarsening_length(image.shape[0],sigma),edgecolor='k',facecolor='w')
kemstem.util.coarsening_marker(ax[2],kemstem.fourier.coarsening_length(image.shape[0],sigma),edgecolor='k',facecolor='w',zorder=2)

# Strain mapping
Finally, strain can be mapped by calculating the phase gradient and looking at its variation along the lattice vector of interest (and normalized to the length of this vector). Artifacts can occur due to wrapping of the phase, adjusting the `mask_threshold` argument may help mitigate them. `sv` is a visualization parameter setting the saturation point of the strain map.

In [ ]:
# strain
sv = .05

ref_x = peaks_ref[peak_index,1]-image.shape[0]/2. 
ref_y = peaks_ref[peak_index,0]-image.shape[0]/2. 
eps_par, eps_trans = kemstem.fourier.phase_to_strain(phase, ref_x, ref_y, mask_threshold=1.)
fig,ax = plt.subplots(1,2,constrained_layout=True,figsize=(8,4),sharex=True,sharey=True)
ax[0].matshow(image,cmap='gray')
ax[0].axis('off')
ax[1].matshow(eps_par,cmap='bwr',vmin=-sv,vmax=sv)
ax[1].axis('off')